## Sample Loops 
### Do not run this! These are just samples you can repurpose.
### T-SQL, PowerShell

In [0]:
--SQLCMD
--:setvar rpl localhost
--:setvar ag AG1

USE [master]
go
if exists (select 1 from sys.availability_databases_cluster inner join sys.databases on database_name = name)
begin
	declare @sql table (smt nvarchar(max))
	declare @cnt int, @smt nvarchar(max)

	insert @sql 
	select 'ALTER AVAILABILITY GROUP [$(ag)] REMOVE DATABASE ['+name+'];' 
	from sys.databases where database_id>4 and name not like 'dist%' and name <> 'Claims' --in ('$(dbs)')

	select @cnt=count(*) from @sql

	while @cnt>0
	begin 
		select top 1 @smt=smt from @sql --order by smt; 
		print @smt; exec sp_executesql @smt	
		delete from @sql where @smt=smt
		set @cnt = @cnt - 1
	end
end 
go

In [0]:
--all else. DONOT RUN THIS!!!
declare @drop nvarchar(max)=null, @cnt int

select @cnt=count(*) from sys.databases where database_id >4 and name not like 'distribution%'

while @cnt>0
begin 
	select top 1 @drop='drop database ['+name+']' from sys.databases where database_id>4 and name not like 'distribution%'
	exec (@drop)
	select @cnt=count(*) from sys.databases where database_id >4 and name not like 'distribution%'
end

In [0]:
declare @name nvarchar(max)=null, @sql nvarchar(max), @count int
select @count=count(*) from sys.databases where state <>0

while @count>0
begin 
	select top 1 @name = name from sys.databases where state <>0
	set @sql = N'restore database ['+@name+'] with recovery'
	--print @sql; 
	exec sp_executesql @sql;
	select @count=count(*) from sys.databases where state <>0
end

In [0]:
EXEC master.dbo.sp_MSforeachdb @command1 = N'
if ''?'' not in (''master'',''model'',''msdb'',''tempdb'',''distribution'',''distribution1'',''distribution2'',''SomeReadonlyDB'',''Skippa'') 
begin 
	exec ?.dbo.sp_changedbowner @loginame = sa, @map = false 
end'
go

PowerShell

In [0]:
%powershell.exe
$Src = 'oldserver'
$Dst = 'newserver1','newserver2'
foreach ($d in $Dst)
{
    Start-DbaMigration -Source $Src -Destination $d  -DisableJobsOnDestination -Exclude Databases -Force
    Copy-DbaDatabaseMail -Source $Src -Destination $d -force
	Copy-DbaAgentOperator -Source $Src -Destination $d -force
	Copy-DbaAgentAlert -Source $Src -Destination $d -force
	Copy-DbaLogin -Source $Src -Destination $d -force
	Copy-DbaLinkedServer -Source $Src -Destination $d -force
    Copy-DbaExtendedEvent -Source $Src -Destination $d -force
    Copy-DbaAgentJob -Source $Src -Destination $d -DisableOnDestination -Force |ft -AutoSize
    Copy-DbaServerAudit -Source $Src -Destination $d -force
    Copy-DbaServerAuditSpecification -Source $Src -Destination $d -force
}